In [16]:
#Import Packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import math
import networkx as nx

#
params = {
"WLSACCESSID": '3',
"WLSSECRET": '2bd3f',
"LICENSEID": 24,
}
env = gp.Env(params=params)

#Import data
pop = pd.read_csv('ID.population')

#Separte edge numbers by spaces
edges = pd.read_csv('ID.dimacs', sep=' ')
edges = edges.drop('p',  axis = 1)

#Initial networkx Graph
G = nx.Graph()
#Remove last row that is not important
edges = edges.iloc[:-1]
#Ensure each edge is a number
edges = edges.apply(pd.to_numeric, errors = 'coerce')
#Make edges a list of tuples
edges = list(edges.itertuples(index=False, name=None))
#Add edges to graph
G.add_edges_from(edges)

#Find Total Population
total = pop['total pop = 1829106'].sum()
print("Total Population: ", total)

#Create Tolerances for population disparity assuming a tolerance of 1% 
U = math.ceil(total/2*1.005)
L = math.floor(total/2*0.995)
print("Upper Boundary: " ,U)
print("Lower Boundary: ", L)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2412244
Academic license 2412244 - for non-commercial use only - registered to cc___@ttu.edu
Total Population:  1839106
Upper Boundary:  924151
Lower Boundary:  914955


In [17]:

#Create the range of counties c
c = range(0,44)
k = 44

#Create the range of districts j
j = range(0,2)
n = 2

#Create Model
m = gp.Model(env=env)

#Add Variables
#Xcj = 1 if county c is assigned to county j and 0 otherwise
x = m.addVars(c, j, vtype = GRB.BINARY)

#Eij is 1 if the edge is cut and 0 otherwise
e = m.addVars(c, c, vtype = GRB.BINARY)

#Set Objective Function
#Minimize the amount of cut edges
m.setObjective(gp.quicksum(e[i, j] for (i, j) in G.edges()), GRB.MINIMIZE)

#Each county can only be assigned one district
m.addConstrs(gp.quicksum(x[u,v] for v in range(n)) == 1 for u in range(k))

#Each Districts population should be between U and L
m.addConstrs(gp.quicksum(x[u,v]*pop['total pop = 1829106'][u] for u in range(k)) <= U for v in range(n))
m.addConstrs(gp.quicksum(x[u,v]*pop['total pop = 1829106'][u] for u in range(k)) >= L for v in range(n))

#An edge is cut if county i and j are assigned to a different district
m.addConstrs(x[i, v] - x[j, v] <= e[i, j] for (i, j) in G.edges() for v in range(n))

#Any edge not in the G.edges is 0 (Likely unneeded)
m.addConstrs(e[i, j] == 0 for i in range(k) for j in range(k) if (i, j) not in G.edges())

#Optimize the Model
m.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: AMD Ryzen 9 6900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2412244 - for non-commercial use only - registered to cc___@ttu.edu
Optimize a model with 1984 rows, 2024 columns and 2608 nonzeros
Model fingerprint: 0x7df69647
Variable types: 0 continuous, 2024 integer (2024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+05]
Found heuristic solution: objective 57.0000000
Presolve removed 1778 rows and 1878 columns
Presolve time: 0.00s
Presolved: 206 rows, 146 columns, 700 nonzeros
Variable types: 0 continuous, 146 integer (146 binary)

Root relaxation: objective 0.000000e+00, 78 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bound

In [18]:
#Print solution
for v in range(n):
    assigned = [u for u in range(k) if x[u, v].x > 0.5] 
    print(f"Counties assigned to District", v, ":", assigned)

#Print the Objective Value
print("Optimal Cut Edges: ", m.objval)

Counties assigned to District 0 : [1, 4, 6, 7, 9, 13, 14, 17, 23, 24, 31, 36, 37, 39, 42]
Counties assigned to District 1 : [0, 2, 3, 5, 8, 10, 11, 12, 15, 16, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 38, 40, 41, 43]
Optimal Cut Edges:  11.0
